In [56]:
# Import ML libraries
import pycaret
import xgboost

# Snowpark for Python
from snowflake.snowpark.session import Session
# import snowflake.snowpark.types as T
# import snowflake.snowpark.functions as F
from snowflake.snowpark.functions import datediff, to_date, col, expr

# Import Misc
import json
import pandas as pd


In [57]:
# import psutil
# def cpu_count_patched(logical=False):
#     import os
#     return os.cpu_count()
# psutil.cpu_count = cpu_count_patched

In [58]:
# Create Snowflake Session object
connection_parameters = json.load(open("connection.json"))

session = Session.builder.configs(connection_parameters).create()

In [59]:

# tableName = 'information_schema.packages'
# dataframe = session.table(tableName).filter(F("language") == 'python')

# session.use_database("HACKATHON_GONEPHISHING")

tableName = 'PURCHASE_ORDER_HISTORY'
dataframe = session.table(tableName)

# Correcting the datediff usage with 'day' as the first argument
dataframe = dataframe.withColumn("target_feature",
                                    datediff('day', col("DELIVERY_DATE_ML"), col("FIRST_GR_POSTING_DATE_ML")))

# Assuming the rest of your columns are correct and relevant
filtered_dataframe = dataframe.select(
    col("PURCHASE_DOCUMENT_ITEM_ID"), # ID for purchase order
    col("VENDOR_ID"),                 # ID of the vendor "we" are purchasing from
    col("POSTAL_CD"),                 # postal code associated w company code ID
    col("PLANT_ID"),                  # ID of plant making purchase
    col("MATERIAL_ID"),               # ID of material being purchase
    col("MRP_TYPE_ID"),               # determined if material is reordered manually or automatically
    col("COMPANY_CODE_ID"),           # copmany w/in INVISTA making purchase
    col("SUB_COMMODITY_DESC"),        # description of sub commodity
    col("INBOUND_DELIVERY_ID"),       # ID for delivery
    col("INBOUND_DELIVERY_ITEM_ID"),  # ID of item w/in delivery
    col("CREATE_DATE_ML"),            # 
    col("DELIVERY_DATE_ML"),          # Delivery date from Purchase Order
    col("FIRST_GR_POSTING_DATE_ML"),  # expected delivery date
    col("REQUESTED_DELIVERY_DATE_ML"),#
    col("PLANNED_DELIVERY_DAYS"),     #
    col("target_feature")             # 
    
)

# Print a sample of the filtered dataframe to standard output.
filtered_dataframe.show()

# Optionally, you might want to filter rows based on some conditions
# Example: Filtering out rows where FIRST_GR_POSTING_DATE_ML is NULL
filtered_dataframe = filtered_dataframe.filter(col("FIRST_GR_POSTING_DATE_ML").is_not_null())


# Show the DataFrame after filtering
filtered_dataframe.show()


# Print a sample of the dataframe to standard output.
# dataframe.show()

SnowparkSQLException: (1304): 01b2b6ce-0002-295b-0048-5a0700185d6a: 002003 (42S02): SQL compilation error:
Object 'PURCHASE_ORDER_HISTORY' does not exist or not authorized.